# House price prediction using Linear Regression

# Reading the dataset

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
import pandas as pd
A = pd.read_csv("C:/Users/Shruti/Downloads/training_set (2).csv")

In [4]:
pd.set_option('display.max_rows',None)

In [5]:
A.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [6]:
#A.isna().sum()

# Missing Data Treatment

In [7]:
def replacer(df):
    import pandas as pd
    Q=pd.DataFrame(df.isna().sum(),columns=["ct"])
    for i in Q[Q.ct>0].index:
                   if(df[i].dtypes!="object"):
                       x=df[i].mean()
                       df[i]=df[i].fillna(x)

#I have replaced only continuous data because in this dataset values which are Nan are actually 'NO' and not "null"
# Thats why we cnange 'Nan' to 'No' with fillna()

In [8]:
replacer(A)

In [9]:
A.Electrical = A.Electrical.fillna("SBrkr")

In [10]:
A=A.fillna('No')

In [11]:
#A.isna().sum()

# Divide data into X and Y

In [12]:
X=A.drop(labels=['SalePrice'],axis=1)
Y=A[["SalePrice"]]

# Selecting best predictors

In [13]:
#For important continuous columns we have used correlation of SalePrice with all other columns and 
#selected columns with best correlation

In [14]:
A.corr()['SalePrice'].sort_values()

KitchenAbvGr    -0.135907
EnclosedPorch   -0.128578
MSSubClass      -0.084284
OverallCond     -0.077856
YrSold          -0.028923
LowQualFinSF    -0.025606
Id              -0.021917
MiscVal         -0.021190
BsmtHalfBath    -0.016844
BsmtFinSF2      -0.011378
3SsnPorch        0.044584
MoSold           0.046432
PoolArea         0.092404
ScreenPorch      0.111447
BedroomAbvGr     0.168213
BsmtUnfSF        0.214479
BsmtFullBath     0.227122
LotArea          0.263843
HalfBath         0.284108
OpenPorchSF      0.315856
2ndFlrSF         0.319334
WoodDeckSF       0.324413
LotFrontage      0.334901
BsmtFinSF1       0.386420
Fireplaces       0.466929
GarageYrBlt      0.470177
MasVnrArea       0.475241
YearRemodAdd     0.507101
YearBuilt        0.522897
TotRmsAbvGrd     0.533723
FullBath         0.560664
1stFlrSF         0.605852
TotalBsmtSF      0.613581
GarageArea       0.623431
GarageCars       0.640409
GrLivArea        0.708624
OverallQual      0.790982
SalePrice        1.000000
Name: SalePr

In [15]:
imp_con_columns =  ["TotalBsmtSF","GarageArea","GarageCars","GrLivArea","OverallQual"]

In [16]:
#For Categorical columns we used ANOVA to get categorical columns with best relation

In [17]:
def ANOVA(df,con,cat):    
    from pandas import DataFrame
    from statsmodels.formula.api import ols
    rel=con+"~"+cat
    model=ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results=anova_lm(model)
    Q=DataFrame(anova_results)
    a = Q['PR(>F)'][cat]
    return round(a,5)

In [18]:
imp_cat_columns=[]
for i in X.columns:
    if A[i].dtypes=='object':
        pv=ANOVA(A,'SalePrice',i)
        if pv<0.05 :
            print('Sale Price vs',i,'--> ',pv)
            imp_cat_columns.append(i)

Sale Price vs MSZoning -->  0.0
Sale Price vs Alley -->  0.0
Sale Price vs LotShape -->  0.0
Sale Price vs LandContour -->  0.0
Sale Price vs LotConfig -->  0.0
Sale Price vs Neighborhood -->  0.0
Sale Price vs Condition1 -->  0.0
Sale Price vs Condition2 -->  0.04343
Sale Price vs BldgType -->  0.0
Sale Price vs HouseStyle -->  0.0
Sale Price vs RoofStyle -->  0.0
Sale Price vs RoofMatl -->  0.0
Sale Price vs Exterior1st -->  0.0
Sale Price vs Exterior2nd -->  0.0
Sale Price vs MasVnrType -->  0.0
Sale Price vs ExterQual -->  0.0
Sale Price vs ExterCond -->  0.0
Sale Price vs Foundation -->  0.0
Sale Price vs BsmtQual -->  0.0
Sale Price vs BsmtCond -->  0.0
Sale Price vs BsmtExposure -->  0.0
Sale Price vs BsmtFinType1 -->  0.0
Sale Price vs BsmtFinType2 -->  0.0
Sale Price vs Heating -->  0.00075
Sale Price vs HeatingQC -->  0.0
Sale Price vs CentralAir -->  0.0
Sale Price vs Electrical -->  0.0
Sale Price vs KitchenQual -->  0.0
Sale Price vs Functional -->  0.00048
Sale Price vs F

In [19]:
imp_con_columns

['TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']

In [20]:
imp_cat_columns

['MSZoning',
 'Alley',
 'LotShape',
 'LandContour',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

# Preprocessing

In [21]:
X1 = pd.get_dummies(X[imp_cat_columns])

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X2 = pd.DataFrame(ss.fit_transform(X[imp_con_columns]),columns=imp_con_columns)
Xnew = X2.join(X1)
Xnew.head()

,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.459303,0.351000,0.311725,0.370333,0.651479,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1,0.466465,-0.060731,0.311725,-0.482512,-0.071836,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,-0.313369,0.631726,0.311725,0.515013,0.651479,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
3,-0.687324,0.790804,1.650307,0.383659,0.651479,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
4,0.199680,1.698485,1.650307,1.299326,1.374795,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0


# Spliting data into training and test set

In [22]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

# OLS Model

In [23]:
from statsmodels.api import OLS,add_constant
x_const=add_constant(xtrain)
ols=OLS(ytrain,x_const)
model=ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     58.05
Date:                Wed, 15 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:15:59   Log-Likelihood:                -13282.
No. Observations:                1168   AIC:                         2.699e+04
Df Residuals:                     953   BIC:                         2.808e+04
Df Model:                         214                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.151e+04   2836.031      4.059      0.000    5946.440    1.71e+04
TotalBsmtSF            9229.7793   1974.200      4.675      0.000    5355.499    1.31e+04
GarageArea             3449.0797   1836.966      1.878      0.061    -155.886    7054.046
GarageCars             2241.0359   1930.369      1.161      0.246   -1547.229    6029.301
GrLivArea              3.446e+04   2080.897     16.562      0.000    3.04e+04    3.85e+04
OverallQual            8190.6753   1589.663      5.152      0.000    5071.032    1.13e+04
MSZoning_C (all)      -3.646e+04   8570.132     -4.255      0.000   -5.33e+04   -1.96e+04
MSZoning_FV            2.425e+04   6979.943      3.474      0.001    1.05e+04    3.79e+04
MSZoning_RH             1.19e+04   6623.902      1.797      0.073   -1094.248    2.49e+04
MSZoning_RL            8638.0458   3469.586      2.490      0.013    1829.134    1.54e+04
MSZoning_RM            3186.2728   4026.893      0.791      0.429   -4716.329    1.11e+04
Alley_Grvl             5291.4980   3772.872      1.403      0.161   -2112.598    1.27e+04
Alley_No               3903.9894   2595.438      1.504      0.133   -1189.445    8997.424
Alley_Pave             2316.5399   3926.559      0.590      0.555   -5389.160       1e+04
LotShape_IR1          -3448.6221   2860.562     -1.206      0.228   -9062.351    2165.107
LotShape_IR2           2419.7447   4152.350      0.583      0.560   -5729.060    1.06e+04
LotShape_IR3           1.606e+04   7163.544      2.241      0.025    1998.610    3.01e+04
LotShape_Reg          -3515.8475   2965.908     -1.185      0.236   -9336.313    2304.618
LandContour_Bnk         301.7798   3536.348      0.085      0.932   -6638.149    7241.708
LandContour_HLS        3099.4691   3644.342      0.850      0.395   -4052.393    1.03e+04
LandContour_Low        7574.4670   4393.256      1.724      0.085   -1047.105    1.62e+04
LandContour_Lvl         536.3114   2354.085      0.228      0.820   -4083.478    5156.101
LotConfig_Corner       4377.9791   3155.607      1.387      0.166   -1814.762    1.06e+04
LotConfig_CulDSac       1.77e+04   3783.780      4.677      0.000    1.03e+04    2.51e+04
LotConfig_FR2         -5854.3963   4276.416     -1.369      0.171   -1.42e+04    2537.883
LotConfig_FR3         -8400.9368   1.04e+04     -0.810      0.418   -2.88e+04     1.2e+04
LotConfig_Inside       3693.8737   2934.625      1.259      0.208   -2065.200    9452.948
Neighborhood_Blmngtn  -8364.4879   7678.113     -1.089      0.276   -2.34e+04    6703.474
Neighborhood_Blueste  -3941.3152   2.36e+04     -0.167      0.867   -5.02e+04    4.23e+04
Neighborhood_BrDale    7265.0198   8884.412      0.818      0.414   -1.02e+04    2.47e+04
Neighborhood_BrkSide   3101.9139   5481.871      0.566      0.572   -7656.018    1.39e+04
Neighborhood_ClearCr  -

# Delete columns on the basis of Adj R Squared Value ( Backward Elimination)

In [111]:
Q = pd.DataFrame(model.pvalues,columns=["pval"])
Q = Q.drop("const")
x = Q.pval.max()
col_to_drop = Q[Q.pval == x].index[0]
print("col =", col_to_drop)
Xnew = Xnew.drop(labels=[col_to_drop],axis=1)


from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,random_state=21,test_size=0.2)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model  = ols.fit()

model.summary()


col = HeatingQC_Fa


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.928
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     83.34
Date:                Wed, 15 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:17:44   Log-Likelihood:                -13285.
No. Observations:                1168   AIC:                         2.689e+04
Df Residuals:                    1010   BIC:                         2.769e+04
Df Model:                         157                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.252e+04   6956.265      1.800      0.072   -1129.559    2.62e+04
TotalBsmtSF            9356.3449   1715.562      5.454      0.000    5989.870    1.27e+04
GarageArea             3399.1456   1712.925      1.984      0.047      37.846    6760.445
GarageCars             2272.1870   1824.947      1.245      0.213   -1308.935    5853.309
GrLivArea              3.374e+04   1762.986     19.136      0.000    3.03e+04    3.72e+04
OverallQual            8158.5366   1453.011      5.615      0.000    5307.271     1.1e+04
MSZoning_C (all)      -3.694e+04   8016.358     -4.608      0.000   -5.27e+04   -2.12e+04
MSZoning_FV            2.224e+04   4225.775      5.262      0.000    1.39e+04    3.05e+04
MSZoning_RH            1.278e+04   6257.512      2.043      0.041     504.112    2.51e+04
MSZoning_RL            9394.8287   3212.683      2.924      0.004    3090.530    1.57e+04
MSZoning_RM            5047.9914   3384.347      1.492      0.136   -1593.164    1.17e+04
LotShape_IR1          -5384.9191   4392.194     -1.226      0.220    -1.4e+04    3233.952
LotShape_IR3           1.465e+04   9393.010      1.560      0.119   -3780.960    3.31e+04
LotShape_Reg          -5459.0657   4487.711     -1.216      0.224   -1.43e+04    3347.239
LandContour_Low        6481.7192   5100.882      1.271      0.204   -3527.820    1.65e+04
LotConfig_Corner       4160.7519   3146.494      1.322      0.186   -2013.662    1.03e+04
LotConfig_CulDSac      1.783e+04   3666.633      4.864      0.000    1.06e+04     2.5e+04
LotConfig_FR2         -5854.2810   4209.962     -1.391      0.165   -1.41e+04    2406.992
LotConfig_FR3         -7323.8759      1e+04     -0.730      0.466    -2.7e+04    1.24e+04
LotConfig_Inside       3705.4288   2992.944      1.238      0.216   -2167.672    9578.529
Neighborhood_Blmngtn  -6305.0724   7573.473     -0.833      0.405   -2.12e+04    8556.471
Neighborhood_BrDale    7158.8906   8763.386      0.817      0.414      -1e+04    2.44e+04
Neighborhood_CollgCr  -3289.8575   3184.076     -1.033      0.302   -9538.019    2958.304
Neighborhood_Crawfor   2.158e+04   4746.313      4.546      0.000    1.23e+04    3.09e+04
Neighborhood_Edwards  -1.287e+04   3727.145     -3.454      0.001   -2.02e+04   -5558.098
Neighborhood_Gilbert  -3613.1192   4193.904     -0.862      0.389   -1.18e+04    4616.643
Neighborhood_IDOTRR   -6099.9104   6334.213     -0.963      0.336   -1.85e+04    6329.814
Neighborhood_MeadowV  -1.261e+04   1.01e+04     -1.254      0.210   -3.23e+04    7116.964
Neighborhood_Mitchel   -1.24e+04   4712.657     -2.632      0.009   -2.17e+04   -3156.279
Neighborhood_NAmes    -1.138e+04   3274.166     -3.476      0.001   -1.78e+04   -4956.077
Neighborhood_NWAmes   -6172.9708   4120.360     -1.498      0.134   -1.43e+04    1912.476
Neighborhood_NoRidge   

# Linear Regression Model

In [112]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [113]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lrmodel=lr.fit(xtrain,ytrain)
tr_pred=lrmodel.predict(xtrain)
ts_pred=lrmodel.predict(xtest)

from sklearn.metrics import mean_absolute_error
tr_err=mean_absolute_error(ytrain,tr_pred)
ts_err=mean_absolute_error(ytest,ts_pred) 

In [114]:
tr_err

14402.385273972603

In [115]:
ts_err

81767116410009.06

# LASSO

In [116]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

In [117]:
Q=[]
x=134
for i in range(1,2000,2):
    x=x+1
    x=round(x,4)
    Q.append(x)

In [118]:
#grid_search
tuning_grid={"alpha":Q}
ls=Lasso()
cv=GridSearchCV(ls,tuning_grid,scoring="neg_mean_absolute_error",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 135}

In [119]:
ls=Lasso(alpha=135)
model=ls.fit(xtrain,ytrain)
pred_tr=model.predict(xtrain)
pred_ts=model.predict(xtest)

In [120]:
from sklearn.metrics import mean_absolute_error
tr_err1=mean_absolute_error(ytrain,pred_tr)
ts_err1=mean_absolute_error(ytest,pred_ts)
tr_err1

15980.883195331275

In [121]:
ts_err1

18703.375168317023

# Read Testing data

In [122]:
X2 = pd.read_csv("C:/Users/Shruti/Downloads/testing_set (3).csv")

In [123]:
X2.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


# Missing data

In [124]:
def replacer(df):
    import pandas as pd
    Q=pd.DataFrame(df.isna().sum(),columns=["ct"])
    for i in Q[Q.ct>0].index:
                   if(df[i].dtypes!="object"):
                       x=df[i].mean()
                       df[i]=df[i].fillna(x)

In [125]:
replacer(X2)

In [126]:
X2.Electrical = X2.Electrical.fillna("SBrkr")

In [127]:
X2=X2.fillna('No')

# Preprocessing

In [128]:
from myFuctions import preprocessing
X2new = preprocessing(X2)

In [132]:
X2new.shape

(1459, 292)

In [133]:
Xnew.shape

(1460, 176)

In [136]:
list(Xnew.columns)

['TotalBsmtSF',
 'GarageArea',
 'GarageCars',
 'GrLivArea',
 'OverallQual',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'LotShape_IR1',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_Low',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'Neighborhood_Blmngtn',
 'Neighborhood_BrDale',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_StoneBr',
 'Neighborhood_Veenker',
 'Condition1_Artery',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_RRAe',
 'Condition2_Feedr',
 'Condition2_Norm',
 'Condition2_PosA',
 'Condition2_RRAe',
 'Condition2_RRNn',
 'BldgType_1Fam',
 'BldgType_2fmCon',
 'Bld

In [148]:
#X2new[list(Xnew.columns)]

In [147]:
#adding the missing columns

In [138]:
Q = ['RoofMatl_Roll', 'Condition2_RRAe', 'PoolQC_Fa', 'RoofMatl_Membran', 'GarageQual_Ex', 'MiscFeature_TenC', 'HouseStyle_2.5Fin', 'RoofMatl_ClyTile', 'Exterior1st_ImStucc', 'Condition2_RRNn', 'Heating_OthW', 'RoofMatl_Metal']

In [139]:
for i in Q:
    X2new[i]=0

In [141]:
XF = X2new[list(Xnew.columns)]

In [143]:
pred = model.predict(XF)

# Prediction

In [144]:
finalDF = X2[["Id"]]
finalDF['SalePrice']=pred

In [145]:
finalDF 

,Id,SalePrice
0,1461,103424.670164
1,1462,160984.237913
2,1463,181935.643080
3,1464,188435.428929
4,1465,200626.098651
5,1466,171248.856231
6,1467,165765.653472
7,1468,160458.630343
8,1469,219641.497267
9,1470,113726.274942


In [146]:
finalDF.to_csv("Desktop/housing_solution.csv")